Import Libs

In [ ]:
import pandas as pd

from preprocess_data import clean_text
from model_randomF import model_rf_train
from model_MultinomialNB import model_mnb_train

In [ ]:
# get csv data

def read_Data(path):

    # Define column names
    column_names = ['label', 'text']
    return pd.read_csv(path, delimiter='\t', encoding='latin-1', header=None, names=column_names)

In [ ]:
# read data

data_train = read_Data(r".\data\training_data_lowercase.csv")
data_test = read_Data(r".\data\testing_data_lowercase_nolabels.csv")    

# preview data
print(data_train.head, "\n")
print(data_test.head, "\n")
print("Shape train data:\n", data_train.shape)
print("Shape test data:\n", data_test.shape)

data_train['cleaned_text'] = data_train['text'].apply(clean_text)
data_test['cleaned_text'] = data_train['text'].apply(clean_text)

print(data_train["cleaned_text"].head, "\n")
print(data_test["cleaned_text"].head, "\n")